This script takes exlu feature classes (derived from the Leon County Property Appraiser annual parcel database update) from 2009 onward & converts them into hexagram bins. Separate feature classes are created for the following attributes derived from parcel attributes: residential units, homesteads, non-residential square feet, prior year market value, and prior year property taxes.

To update with new year's data:
- copy the new year's exlu feature class into the '/original_exlu_FCs' project dataset named as 'exlu_{YYYY}'
- if neither City nor USA boundary changed, can just run this script for the one new year; if they did change, then will need to rerun for all years
- update 'cpi_yr' and add the new year's CPI to 'cpi_dict'
- if City and/or USA boundaries did change, project new ones into the same State Plane projection as the exlu; keep same names (TallahasseeLimits_StatePlane' & 'USABound_StatePlane')

IMPORTS, WORKSPACE & DATA

In [1]:
import arcpy
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor

# set initial workspace
arcpy.env.workspace = r"C:\Users\bryantch\Documents\ArcGIS\Projects\hexbin_parcel_analysis\hexbin_parcel_analysis.gdb"

# OK to overwrite previous versions of layers & feature classes
arcpy.env.overwriteOutput = True

### ADD NEW YEAR EACH NOVEMBER
years = ['2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']

### ADD NEW YEAR & INDEX EACH NOVEMBER
# Consumer Price Index history to convert all dollars to current year 
# (https://www.minneapolisfed.org/about-us/monetary-policy/inflation-calculator/consumer-price-index-1913-)
cpi_dict = {'2009':214.5, '2010':218.1, '2011':224.9, '2012':229.6, '2013':233.0, \
            '2014':236.7, '2015':237.0, '2016':240.0, '2017':245.1, '2018':251.1, \
            '2019':255.7, '2020':258.8, '2021':271.0, '2022':292.7}

base_yr = years[0]
final_yr = years[-1] # for calculating current value for market & tax fields

PRE-PROCESS EXLU

In [ ]:
# CLIP BY THE URBAN SERVICE AREA BOUNDARY & SAVE TO A NEW FEATURE CLASS
# all layers in NAD_1983_HARN_StatePlane_Florida_North_FIPS_0903_Feet

for yr in years:

    in_features = f'/original_exlu_FCs/exlu_{yr}'
    clip_features = 'USABound_StatePlane'
    # clip_features = 'TallahasseeLimits__StatePlane' # may alternate USA versus City depending on the application
    out_feature_class = f'/intermediate_parcel_FCs/parcels_{yr}'  

    arcpy.analysis.Clip(in_features, clip_features, out_feature_class)

    print(yr + " complete")

In [ ]:
# ENSURE THE HOMESTEAD FIELD HAS THE SAME NAME FORMAT FOR ALL YEARS
# 2009 uses 'HX{yr}' format, stops in 2010, then starts again 2018-2019; changes to 'HZ' in 2020

for yr in years:

    parcels_FC = f'/intermediate_parcel_FCs/parcels_{yr}'

    new_field_name = 'HX'
    new_field_alias = 'HX'

    # get a list of the fields
    fieldList = arcpy.ListFields(parcels_FC)

    for field in fieldList:
        if field.name.startswith('HX') or field.name.startswith('HZ'):
            arcpy.management.AlterField(parcels_FC, field.name, new_field_name, new_field_alias)

    print(yr + ' complete')

In [ ]:
# CORRECT RESIDENTIAL UNITS FOR NURSING HOMES AS THEY ARE CONSIDERED GROUP QUARTERS IN LATER YEARS & CORRECT 
# OTHER MISTAKES NOTED BY ED YOUNG
# TODO - make sure the parcel number/unit changes are appropriate for all years, not just 2009

for yr in years:

    inTable_corrections = f'/intermediate_parcel_FCs/parcels_{yr}'
    fieldName_corrections = 'resunits'
    expression_corrections = 'calc_corrections(!pattern!, !resunits!, !TAXID!)'
    codeblock_corrections = '''
def calc_corrections(pattern, residentialUnits, taxID):
    if pattern == 'nursing home':
        residentialUnits = 0
    elif pattern == 'nursing facility':
        residentialUnits = 0
    elif pattern == 'clinic':
        residentialUnits = 0
    elif taxID == '2119350000070':
        residentialUnits = 0
    elif taxID == '2136800477790':
        residentialUnits = 0
    elif taxID == '2134206440000':
        residentialUnits = 26
    elif taxID == '2135202110000':
        residentialUnits = 39
    else: 
        residentialUnits = residentialUnits
    return residentialUnits'''

    # calculate the new field
    arcpy.management.CalculateField(inTable_corrections, fieldName_corrections, expression_corrections, "PYTHON3", codeblock_corrections, field_type="DOUBLE")

    print(yr + " complete")

ACCOUNT FOR PARCELS THAT CONVERTED FROM CONDO/TOWNHOUSE WITH COMMON AREAS INTO A SINGLE MULTIFAMILY PARCEL AS THIS CAUSES FALSE GAINS/LOSSES IN THE AFFECTED HEXAGRAM BINS

*switching from arcpy to spacially enabled dataframes as it reduces run time by 5+ minutes per year (1+ hour total)*

In [ ]:
for yr in years:

    # import parcels as dataframe
    path_parcels = fr"C:\Users\bryantch\Documents\ArcGIS\Projects\hexbin_parcel_analysis\hexbin_parcel_analysis.gdb\intermediate_parcel_FCs\parcels_{yr}"
    parcels_df = pd.DataFrame.spatial.from_featureclass(path_parcels)
    print(f'start - {yr}:{parcels_df.shape}')

    # feature classes were previously created for each year affected by apartment conversions; it summarizes attribute data as one feature per apartment complex
    path_mf_corrections = fr'C:\Users\bryantch\Documents\ArcGIS\Projects\hexbin_parcel_analysis\hexbin_parcel_analysis.gdb\multifamily_conversion_FCs\multifam_conversions_{yr}'
    
    if arcpy.Exists(path_mf_corrections):

        # import the summarized apartment feature class as a dataframe
        mf_corrections_df = pd.DataFrame.spatial.from_featureclass(path_mf_corrections)

        # for each converted condo & townhouse development, delete the multiple parcels
        unique_legals = mf_corrections_df.LEGAL1.unique().tolist()
        for legal in unique_legals:
            parcels_to_remove = parcels_df[(parcels_df.LEGAL1 == legal)].index
            parcels_df.drop(parcels_to_remove, inplace=True)

        # add the corrected parcel feature class (previously created in Pro) back into the original parcel layer
        frames = [parcels_df, mf_corrections_df]
        parcels_df = pd.concat(frames)

        # keep only the necessary fields
        fields = ['OBJECTID', 'resunits', 'PYR_MARKET', 'PYR_TAXES', 'HX', 'BASE_SQ_FT', 'AUX_SQ_FT', 'exlanduse', 'homestead', 'SHAPE']
        parcels_df = parcels_df[fields]

        # TODO - TEST THIS WORKS
        # convert all NaN to zeros (this should prevent comparison ('>') errors in the CPI calculations below)
        parcels_df[['PYR_MARKET', 'PYR_TAXES']] = parcels_df[['PYR_MARKET', 'PYR_TAXES']].fillna(0)

        # verify number of rows, columns changed    
        print(f'corrected - {yr}:{parcels_df.shape}')
        
        # write the dataframe back to the gdb
        parcels_df.spatial.to_featureclass( location=path_parcels, 
                                            sanitize_columns=False,
                                            overwrite=True)
    
    else: # for the years that didn't have any conversions

        # keep only the necessary fields
        fields = ['OBJECTID', 'resunits', 'PYR_MARKET', 'PYR_TAXES', 'HX', 'BASE_SQ_FT', 'AUX_SQ_FT', 'exlanduse', 'SHAPE']
        parcels_df = parcels_df[fields]

        # TODO - TEST THIS WORKS
        # convert all NaN to zeros (this should prevent comparison ('>') errors in the field calculations below)
        parcels_df[['PYR_MARKET', 'PYR_TAXES']] = parcels_df[['PYR_MARKET', 'PYR_TAXES']].fillna(0)

        # write the dataframe back to the gdb
        parcels_df.spatial.to_featureclass( location=path_parcels, 
                                            sanitize_columns=False,
                                            overwrite=True)
        
        # verify year was processed
        print(f'no correction - {yr}:{parcels_df.shape}')

CALCULATE NEEDED FIELDS

In [ ]:
# UPDATE MARKET & TAXES FIELDS TO ACCOUNT FOR INFLATION USING MOST CURRENT YEAR DOLLARS
# Past dollars in terms of recent dollars = Dollar amount × Ending-period CPI ÷ Beginning-period CPI
# (https://www.fool.com/investing/general/how-to-calculate-the-real-value-of-money-using-the.aspx)

for yr in years:

    currVal_list = ['PYR_MARKET', 'PYR_TAXES']

    for field in currVal_list:

        inTable_currVal = f'/intermediate_parcel_FCs/parcels_{yr}'
        fieldName_currVal = field
        expression_currVal = f"calc_currVal(!{field}!)"

        codeblock_currVal = f""" 
def calc_currVal({field}):
    for year in cpi_dict:
        if yr == year:
            if {field} > 0: 
                return {field} * cpi_dict['{final_yr}'] / cpi_dict[year]"""

        arcpy.management.CalculateField(inTable_currVal, fieldName_currVal, expression_currVal, "PYTHON3", codeblock_currVal, field_type="DOUBLE")

    print (yr + " complete")

In [ ]:
# CALCULATE NEW FIELD 'homestead'
# with value of "1" if HX is "X" and "0" if else

for yr in years:

    inTable_hmstead = f'/intermediate_parcel_FCs/parcels_{yr}'
    fieldName_hmstead = 'homestead'
    expression_hmstead = 'calc_hmstead_integer(!HX!, !homestead!)'
    codeblock_hmstead = '''
def calc_hmstead_integer(HX, homestead):
    if HX == "X":
        homestead = 1
    else:
        homestead = homestead
    return homestead'''

    # calculate the new field
    arcpy.management.CalculateField(inTable_hmstead, fieldName_hmstead, expression_hmstead, "PYTHON3", codeblock_hmstead, field_type="DOUBLE")

    print(yr + " complete")

In [ ]:
# CALCULATE 'nonResSF' 
# using exlanduse and BASE_SQ_FT+AUX_SQ_FT

for yr in years:

    inTable_nonResSF = f'/intermediate_parcel_FCs/parcels_{yr}'
    fieldName_nonResSF = 'nonResSF'
    expression_nonResSF = 'calc_nonResSF(!exlanduse!, !BASE_SQ_FT!, !AUX_SQ_FT!)'
    codeblock_nonResSF = '''
def calc_nonResSF(exlanduse, baseSF, auxSF):
    nonResSF = 0
    if baseSF == None:
        baseSF = 0
    if auxSF == None:
        auxSF = 0
    if exlanduse in ['Retail', 'Office', 'Warehouse', 'Religious/Non-profit', 'School', 'Motel/Hospital/Clinic', 'Government']:
        nonResSF = baseSF + auxSF
    return nonResSF'''

    # calculate the new field
    arcpy.management.CalculateField(inTable_nonResSF, fieldName_nonResSF, expression_nonResSF, "PYTHON3", codeblock_nonResSF, field_type="DOUBLE")

    print(yr + " complete")

In [ ]:
# DELETE FIELDS USED IN CALCULATIONS BUT NO LONGER NEEDED
for yr in years:

    inTable_delete = f'/intermediate_parcel_FCs/parcels_{yr}'
    fields = ['HX', 'BASE_SQ_FT', 'AUX_SQ_FT', 'exlanduse']

    arcpy.management.DeleteField(inTable_delete, fields, method='DELETE_FIELDS')

    print(yr + " complete")

PLACE THE PARCEL DATA INTO 1-ACRE HEXAGRAM BINS & UPDATE THE FIELDS AS NECESSARY

In [ ]:
for yr in years:

    prior_yr = str(int(yr) - 1)

    # run summarize within tool
    summarized_layer = f'/intermediate_parcel_FCs/parcels_{yr}'
    output_fc = f'/hexBin_FCs/hexBin_{yr}'
    sum_polygons = f'/hexBin_FCs/hexBin_{prior_yr}' # need to use prior year after the initial run b/c SummarizeWithin joins all previous fields not just bin geometry
    sum_fields = [  ['resunits', 'SUM', 'Count'], ['PYR_MARKET', 'SUM', 'Count'], ['PYR_TAXES', 'SUM', 'Count'], \
                    ['homestead', 'SUM', 'Count'], ['nonResSF', 'SUM', 'Count']]
    if yr == base_yr: # this version used only for the first year to create the initial bins
        arcpy.gapro.SummarizeWithin(summarized_layer, output_fc, polygon_or_bin='BIN', bin_type='HEXAGON', \
                                    bin_size='224.27 Feet', sum_shape='ADD_SUMMARY', shape_units='ACRES_US', standard_summary_fields=sum_fields)
    else:
        arcpy.gapro.SummarizeWithin(summarized_layer, output_fc, polygon_or_bin='POLYGON', bin_type='HEXAGON', \
                                    summary_polygons=sum_polygons, sum_shape='ADD_SUMMARY', shape_units='ACRES_US', standard_summary_fields=sum_fields)
        
    # update the analysis field name to include the year
    inTable_cleaning = f'/hexBin_FCs/hexBin_{yr}'
    analysis_fields = ['SUM_resunits', 'SUM_PYR_MARKET', 'SUM_PYR_TAXES', 'SUM_homestead', 'SUM_nonResSF']
    for field in analysis_fields:
        new_field_name = f'{field}_{yr}_n'
        new_field_alias = f'{field}_{yr}_n'
        arcpy.management.AlterField(inTable_cleaning, field, new_field_name, new_field_alias)

    # remove extraneous 'SUM_' prefix
    fieldList_cleaning = arcpy.ListFields(inTable_cleaning)
    for field in fieldList_cleaning:
        if field.name.startswith('SUM_'): 
            new_field_name = field.name.strip('SUM_').lower()
            new_field_alias = field.name.strip('SUM_').lower()
            arcpy.management.AlterField(inTable_cleaning, field.name, new_field_name, new_field_alias)

    # calculate annual change for each variable (could be done in JS/browser but this should enhance performance)
    if yr != base_yr:
        analysis_fields2 = ['resunits', 'pyr_market', 'pyr_taxes', 'homestead', 'nonressf']
        inTable_annual_change = f'/hexBin_FCs/hexBin_{yr}'
        codeblock_annual_change = '''
def calc_annual_change (prior_yr_field, yr_field):
    annual_change = yr_field - prior_yr_field
    return annual_change'''

        for field in analysis_fields2:
            fieldName_annual_change = f'{field}_{yr}_A'
            expression_annual_change = f'calc_annual_change(!{field}_{prior_yr}_n!, !{field}_{yr}_n!)'
            arcpy.management.CalculateField(inTable_annual_change, fieldName_annual_change, expression_annual_change, 'PYTHON', codeblock_annual_change, field_type="DOUBLE")

    # calculate total change for each variable
    if yr != base_yr:
        inTable_total_change = f'/hexBin_FCs/hexBin_{yr}'
        codeblock_total_change = '''
def calc_total_change (base_yr_field, yr_field):
    total_change = yr_field - base_yr_field
    return total_change'''


        for field in analysis_fields2:
            fieldName_total_change = f'{field}_{yr}_T'
            expression_total_change = f'calc_total_change(!{field}_{base_yr}_n!, !{field}_{yr}_n!)'
            arcpy.management.CalculateField(inTable_total_change, fieldName_total_change, expression_total_change, 'PYTHON', codeblock_total_change, field_type="DOUBLE")

    print(yr + " complete")

SPLIT INTO SEPARATE LAYERS FOR EACH VARIABLE (should result in better web application performance)

In [4]:
# import the final year hexbin feature class as a spatially enabled dataframe
final_hex_path = f'C:/Users/bryantch/Documents/ArcGIS/Projects/hexbin_parcel_analysis/hexbin_parcel_analysis.gdb/hexBin_FCs/hexBin_{final_yr}'
final_hex_df = pd.DataFrame.spatial.from_featureclass(final_hex_path)

# make dataframes for each attribute field & save new dataframes to the gdb

attribute_FCs = ['resunits', 'pyr_market', 'pyr_taxes', 'homestead', 'nonressf']

for attribute in attribute_FCs:

    # filter to keep only needed columns (regex tests for the strings or substrings in column names)
    attribute_df = final_hex_df.filter(regex= f'OBJECTID|SHAPE|{attribute}')

    # drop the extraneous OBJECTID columns but keep the most current one
    keep_cols = [col for col in attribute_df.columns if not 'OBJECTID' in col or col=="OBJECTID"]
    attribute_df = attribute_df[keep_cols]

    # round to the first decimal point
    decimalDict = {col:1 for col in attribute_df.columns if col.startswith(f'{attribute}')}
    attribute_df = attribute_df.round(decimalDict)

    # drop any rows with all zeroes in attribute fields ('OBJECTID' is also int64 type so need to subtract it off)
    attribute_df = attribute_df[((attribute_df.sum(axis=1, numeric_only=True) - attribute_df["OBJECTID"]) != 0)]

    # save to gdb
    attribute_fc_path = f'C:/Users/bryantch/Documents/ArcGIS/Projects/hexbin_parcel_analysis/hexbin_parcel_analysis.gdb/attribute_FCs/{attribute}_hexbins'
    attribute_df.spatial.to_featureclass(location=attribute_fc_path, 
                                         sanitize_columns=False,
                                         overwrite=True) 

    print (attribute + ' complete')                

resunits complete
pyr_market complete
pyr_taxes complete
homestead complete
nonressf complete


MAKE POLYGON TO POINT LAYER FOR ALL LAYERS TO BE USED FOR CLUSTERING LABELS

In [6]:
# if the feature class already exists, delete it before adding the new one
for attribute in attribute_FCs:
    out_feature_class = f'/attribute_FCs/{attribute}_points'
    in_features = f'/attribute_FCs/{attribute}_hexbins'
    if arcpy.Exists(out_feature_class):
        arcpy.Delete_management(out_feature_class)
        arcpy.management.FeatureToPoint(in_features, out_feature_class, point_location='CENTROID')

        print (attribute + ' complete')

resunits complete
pyr_market complete
pyr_taxes complete
homestead complete
nonressf complete
